Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [1]:
!pip install -q -U transformers
!pip install -q datasets

In [2]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import  TrainingArguments, Trainer, DataCollatorWithPadding

In [3]:
corpus = load_dataset('merionum/ru_paraphraser')

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [5]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
label_list = sorted(set(corpus['train']['class']))
label_list

['-1', '0', '1']

In [8]:
labels2id = { key:id for id, key in enumerate(label_list)}
labels2id

{'-1': 0, '0': 1, '1': 2}

In [9]:
id2labels = { id:key for id, key in enumerate(label_list)}
id2labels

{0: '-1', 1: '0', 2: '1'}

In [10]:
def tokenize_and_align_labels(tokenizer, labels2id):
    def tokenize_and_align_labels_(examples):
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_

In [11]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(corpus['train'][20:24])
tokenized_input

{'input_ids': [[0, 18449, 1068, 1951, 169066, 59, 42136, 59392, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2, 2, 6, 144048, 145158, 184643, 18449, 28196, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2], [0, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 7329, 155333, 24863, 5, 2, 2, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 534, 11591, 149673, 5, 2], [0, 417, 14100, 695, 1200, 49, 30037, 1269, 326, 3699, 108854, 1857, 56869, 90229, 546, 209, 9561, 5, 2, 2, 417, 68642, 69, 53177, 29, 108854, 1857, 47745, 49, 14100, 695, 1200, 90229, 546, 209, 9561, 5, 2], [0, 72085, 138109, 87830, 29, 23262, 2294, 90569, 11974, 24744, 59, 161679, 8568, 5, 2, 2, 69242, 64393, 41976, 4560, 1560, 182076, 29, 29230, 23262, 49, 72085, 138109, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [12]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][1])
print(tokens)

['<s>', '▁В', '▁нового', 'д', 'нюю', '▁ночь', '▁во', '▁Франции', '▁со', 'ж', 'гли', '▁больше', '▁тысячи', '▁машин', '.', '</s>', '</s>', '▁В', '▁нового', 'д', 'нюю', '▁ночь', '▁во', '▁Франции', '▁со', 'ж', 'гли', '▁11', '93', '▁автомобиля', '.', '</s>']


In [13]:
corpus['train'][20]

{'id': '21',
 'id_1': '3',
 'id_2': '4116',
 'text_1': 'Чечня попросила националистов составить кодекс поведения кавказцев.',
 'text_2': 'Националисты помогут Чечне составить кодекс поведения кавказцев.',
 'class': '0'}

In [14]:
tokenized_datasets = corpus.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1924
    })
})

In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list), is_decoder=True)

model.config.id2label = id2labels
model.config.label2id = labels2id

In [17]:
metric = load_metric("accuracy")

<ipython-input-17-21f3ecd7ee32>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate

In [21]:
batch_size = 16
epochs = 4

args = TrainingArguments("paraphras",
                         evaluation_strategy="epoch",
                         learning_rate=2e-6,
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size,
                         num_train_epochs=epochs,
                         weight_decay=0.05,
                         save_strategy='no',
                         report_to='none')

In [22]:
trainer = Trainer(model,
                  args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  data_collator=data_collator,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

In [23]:
%%time

trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.062728,0.404366
2,1.105200,1.056083,0.404366
3,1.083200,1.058093,0.404366
4,1.081700,1.058970,0.404366


CPU times: user 17min 19s, sys: 2min 1s, total: 19min 20s
Wall time: 19min 46s


TrainOutput(global_step=1808, training_loss=1.0894410293714134, metrics={'train_runtime': 1186.6266, 'train_samples_per_second': 24.361, 'train_steps_per_second': 1.524, 'total_flos': 2425865579130252.0, 'train_loss': 1.0894410293714134, 'epoch': 4.0})

In [ ]:
example = tokenized_datasets["test"][40:80]

In [25]:
tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')

with torch.no_grad():
    outputs = model(**tokens)

predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]

In [26]:
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'],
                          'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0,0
1,Источник: Якунин озвучил свою месячную зарплат...,Якунин назвал размер своей зарплаты,0,0
2,Apple получила патент на социальную сеть,Apple получила патент на собственную соцсеть,1,0
3,Подвесной потолок обрушился в аэропорту Стамбу...,Четверо пострадали при обрушении подвесного по...,1,0
4,Россия представляет принципиально новый танк,Вашингтон пригрозил России новыми санкциями,-1,0
5,Генсек ООН и премьер Украины обсудили минские ...,Генсек ООН и премьер-министр Украины обсудили ...,1,0
6,Результаты выборов в Великобритании укрепили фунт,Стали известны окончательные результаты выборо...,-1,0
7,Николич: Сербия не говорила о нежелании строит...,Президент Сербии опроверг свои слова об отказе...,0,0
8,СМИ сообщают об обысках в Минкульте по делу о ...,Минкультуры отчиталось перед следствием по дел...,-1,0
9,Любляна отпразднует День Победы вместе с Москвой,Путеводитель по Дню Победы: как провести 9 мая...,-1,0
